In [1]:
import pandas as pd
import numpy as np
from prettytable import PrettyTable

In [2]:
app_files = ["console_2018-12-27_12-16-52.log", "console_2018-12-27_12-45-57.log", 
         "console_2018-12-27_13-49-15.log", "console_2018-12-27_14-34-15.log"]
rl_files = ["run_length_2018-12-27_12-16-18.log", "run_length_2018-12-27_12-53-01.log", 
            "run_length_2018-12-27_13-49-32.log", "run_length_2018-12-27_14-34-55.log"]

In [3]:
objects = {"[0, 0, 0, 0, 0, 0]":"none",
           "[1, 0, 0, 0, 0, 0]":"cards", 
           "[0, 1, 0, 0, 0, 0]":"dice", 
           "[0, 0, 1, 0, 0, 0]":"key",
           "[0, 0, 0, 1, 0, 0]":"map", 
           "[0, 0, 0, 0, 1, 0]":"phone", 
           "[0, 0, 0, 0, 0, 1]":"ball"}

In [4]:
def read_app_file(file_name):
    with open("data/" + file_name, "r") as f:
        lines  = f.readlines()
        data = lines[2:-2]
        data_list = []
        for line in data:
            object_time = line.split("$1")[1].split("-")[1].strip().split(",")
            if object_time[0] == 'cancel':
                data_list.pop()
            else:
                object_time[1] = float(object_time[1].strip())
                data_list.append(object_time)
    return data_list

In [5]:
def read_rl_file(file_name):
    with open("data/" + file_name, "r") as f:
        lines  = f.readlines()
        hit_scan_list = []
        for line in lines:
            parts = line.split(";")
            time = float(parts[1].split("-")[1])
            glass1 = parts[2].split("-")[1]
            glass2 = parts[3].split("-")[1]
            hit_scan_list.append((time, objects.get(glass1), objects.get(glass2)))
    return hit_scan_list

In [6]:
def create_ds(app_file, rl_file):
    app_data = read_app_file(app_file)
    hsl_data = read_rl_file(rl_file)        

    ot_rl = []
    last_e_ind=0
    for i, ot in enumerate(app_data):
        s_time = ot[1]

        if i != 0:
            s_ind = last_e_ind
        else:
            for j, hsl in enumerate(hsl_data):
                if hsl[0] > s_time:
                    s_ind = j-1
                    break

        if i != (len(app_data) -1):
            e_time = app_data[i+1][1]
            e_ind = 0
            for j, hsl in enumerate(hsl_data[s_ind:]):
                if hsl[0] > e_time:
                    e_ind = s_ind + j
                    break
            rl = hsl_data[s_ind:e_ind]
            last_e_ind = e_ind
        else:
            rl = hsl_data[s_ind:]
        
        new_ot = ot.copy()
        new_ot.append(rl)
        ot_rl.append(new_ot)
    return ot_rl   

In [7]:
def process_ds(ot_rl_list):
    new_ot_rl = []
    for ot_rl in ot_rl_list:
        obj = ot_rl[0]
        rl_data = ot_rl[2]
        timeseries = []
        detections = []
        for rl in rl_data:
            timeseries.append(rl[0])
            detection = 0
            if obj == rl[1] and obj == rl[2]:
                detection = 1
            detections.append(detection)
        rld = {"ts":timeseries, "hits":detections}
        new_data = (obj, ot_rl[1], rld)
        new_ot_rl.append(new_data)
    return new_ot_rl

In [8]:
tot_ot_rl = []
for i in range(len(app_files)):
    ot_rl = create_ds(app_files[i], rl_files[i])
    tot_ot_rl.extend(ot_rl)

In [9]:
new_ot_rl = process_ds(ot_rl)

In [10]:
class RunLengthFilter:
    
    def __init__(self, ot_rl_list, true_look_size, true_look_delta):
        self.data = ot_rl_list
        self.true_size = true_look_size
        self.true_delta = true_look_delta
        
    def rl_filter(self, window_size, num_ones):
        
        df = {'object':[], 'app_time':[], 'look_time':[]}
        diff = window_size - num_ones
        
        for ot_rl in self.data:
            df['object'].append(ot_rl[0])
            df['app_time'].append(ot_rl[1])
            rl_data = ot_rl[2]
            ts = rl_data['ts']
            hits = rl_data['hits']
            look_time = 0
            
            for i in range(-diff, len(ts)):
                if i<0:
                    s_idx = 0
                else:
                    s_idx = i
                e_idx = i + window_size
                if sum(hits[s_idx:e_idx]) == num_ones:
                    look_time = ts[e_idx]
                    break
            df['look_time'].append(look_time)
        
        df = pd.DataFrame(df)
        df['latency'] = df['look_time'] - df['app_time']
        return df
    
    def determine_true_look(self):
        for ot_rl in self.data:
            df['object'].append(ot_rl[0])
            df['app_time'].append(ot_rl[1])
            rl_data = ot_rl[2]
            ts = rl_data['ts']
            hits = rl_data['hits']
            look_time = 0
            
            for i in range(-diff, len(ts)):
                if i<0:
                    s_idx = 0
                else:
                    s_idx = i
                e_idx = i + window_size
                if sum(hits[s_idx:e_idx]) == num_ones:
                    look_time = ts[e_idx]
                    break
            df['look_time'].append(look_time)

In [11]:
rl3 = RunLengthFilter(3, 3)
rl4 = RunLengthFilter(4, 4)

In [12]:
rl3_df = rl3.rl_filter(new_ot_rl)
rl4_df = rl4.rl_filter(new_ot_rl)

In [13]:
def summary(df_rl):
    rl_dict = {}
    for obj in df_rl["object"].unique():
        df = df_rl[df_rl["object"] == obj]
        m = round(np.mean(df["latency"]), 3)
        s = round(np.std(df["latency"]), 3)
        rl_dict[obj] = [m, s]
    return rl_dict

In [14]:
rl3_dict = summary(rl3_df)
rl4_dict = summary(rl4_df)

In [15]:
table = PrettyTable(["Object", "RL3 Mean", "RL3 Std", "RL4 Mean", "RL4 Std"])
for obj in rl3_df["object"].unique():
    table.add_row([obj, rl3_dict[obj][0], rl3_dict[obj][1], rl4_dict[obj][0], rl4_dict[obj][1]])
print(table)

+--------+----------+---------+----------+---------+
| Object | RL3 Mean | RL3 Std | RL4 Mean | RL4 Std |
+--------+----------+---------+----------+---------+
|  ball  | 1110.421 | 307.811 | 1114.579 | 307.112 |
| cards  | 1061.158 | 144.646 | 1063.526 | 144.225 |
|  dice  | 987.632  | 153.516 | 998.842  | 150.375 |
|  key   | 896.944  |  95.317 | 900.111  |  95.344 |
|  map   | 851.222  | 108.044 |  854.0   | 109.271 |
+--------+----------+---------+----------+---------+


In [16]:
rl3_df['look_time'][1]

1545950446487.0

In [17]:
rl4_df['look_time'][1]

1545950446488.0